# SISTEMA DE RECOMENDACIÓN DE EJERCICIOS DE PROGRAMACIÓN

## 1. COMPRENCIÓN DEL PROBLEMA

Dentro de la Universidad Andrés Bello, los estudiantes de carreras de ingeniería deben cursar la asignatura de "Introducción a la Programación", ya que necesitan tener conocimientos básicos, medios o avanzados de programación, dependiendo de la carrera que estén cursando y en qué año van.

La problemática surge cuando ciertos estudiantes reprueban o sacan notas mínimas en las pruebas y trabajos de la asignatura. Siendo esta la primera asignatura de programación de ciertas carreras, puede marcar seriamente el desempeño del estudiante en el futuro, llegando a provocar que algunos tengan deficiencias en conocimientos esenciales para sus carreras.

Además, los estudiantes a menudo carecen de orientación personalizada para su aprendizaje, lo que puede resultar en un bajo rendimiento en los métodos de estudio, ocasionando un bajo conocimiento o baja nota en la asignatura.

Por tanto, este proyecto busca desarrollar e implementar un sistema de recomendación de ejercicios de programación para estudiantes universitarios de ingeniería de la Universidad Andrés Bello. Este sistema personalizará las sugerencias de ejercicios según el nivel de habilidad, preferencias individuales y objetivos de aprendizaje de cada estudiante.

## 2. COMPRENCIÓN DE LOS DATOS

### CATALOGO

| Campo         | Descripción                                                                                                                                               | Tipo  |   Valores     | 
|---------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|-------|---------------|
| hito          | Conjunto de ejercicios                                                                                                                                    | int   | [1,2,3,4]     |
| oid           | Identificador del ejercicio                                                                                                                               | str   | *             |
| nombre        | Nombre del ejercicio                                                                                                                                      | str   | *             |
| s1            | Nivel 4 en Algoritmos - Algoritmos Recursivos                                                                                                             | int   | binario [0,1] |
| s2            | Nivel 3 en Algoritmos - Algoritmos que combinan varias estructuras de datos                                                                               | int   | binario [0,1] |
| s3            | Nivel 2 en Algoritmos - Algoritmos que presentan información utilizando una estructura de datos                                                           | int   | binario [0,1] |
| s4            | Nivel 1 en Algoritmos - Algoritmos que no requieren una estructura de datos                                                                               | int   | binario [0,1] |
| k1            | Conocimientos en definir una clase con atributos y métodos.                                                                                               | int   | binario [0,1] |
| k2            | Conocimientos en agregar, quitar y buscar elementos dentro de una estructura de datos (cadenas o listas).                                                 | int   | binario [0,1] |
| k3            | Conocimientos en definir un bucle y determinar sus características en función del problema.                                                               | int   | binario [0,1] |
| k4            | Conocimientos en definir variables, usar operadores de comparación, construir expresiones condicionales, etc.                                             | int   | binario [0,1] |
| skill         | un escalar que indica la habilidad del estudiante                                                                                                         | int   | (0, 12)       |
| knowledge     | un escalar que indica el conocimiento del estudiante                                                                                                      | int   | (0, 12)       |
| complexity    | un escalar que indica la complejidad, número binario de 8 bits resultante de unir s1,s2,s3,s4,k1,k2,k3,k4                                                 | int   | binario [0,1] |
| item          | Código alfabético asociado a la complejidad, combina habilidad (4 bits en ASCII) con conocimiento (4 bits en ASCII): 0->a,1->b,2->c, etc.                 | str   | *             |


### DATA_A

| Campo         | Descripción                                                                                                                              | Tipo  | Valores    | 
|---------------|------------------------------------------------------------------------------------------------------------------------------------------|-------|------------|
| rut           | Número de identificación ficticio del estudiante                                                                                         | int   | continuo   |
| usuario       | anonimo                                                                                                                                  | *     | *          |
| correo        | anonimo                                                                                                                                  | *     | *          |
| nombre        | anonimo                                                                                                                                  | *     | *          |
| nrc           | nrc del curso en donde el estuiante vio la asignatura                                                                                    | int   | continuo   |
| profesor      | anonimo                                                                                                                                  | *     | *          |
| i-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| f-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| t-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| r-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| p-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| s-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| l-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| op-(1, 6)     | anonimo                                                                                                                                  | *     | *          |
| sv-(1, 6)     | anonimo                                                                                                                                  | *     | *          |
| lt            | anonimo                                                                                                                                  | *     | *          |
| tt            | anonimo                                                                                                                                  | *     | *          |
| pt            | anonimo                                                                                                                                  | *     | *          |
| game_score    | anonimo                                                                                                                                  | *     | *          |
| score         | anonimo                                                                                                                                  | *     | *          |
| score_a       | anonimo                                                                                                                                  | *     | *          |
| score_p       | anonimo                                                                                                                                  | *     | *          |
| score_d       | anonimo                                                                                                                                  | *     | *          |
| score_s       | anonimo                                                                                                                                  | *     | *          |
| tema2         | anonimo                                                                                                                                  | *     | *          |
| tema3         | anonimo                                                                                                                                  | *     | *          |
| tema4         | anonimo                                                                                                                                  | *     | *          |
| tema5         | anonimo                                                                                                                                  | *     | *          |
| tema6         | anonimo                                                                                                                                  | *     | *          |
| tema7         | anonimo                                                                                                                                  | *     | *          |
| tema8         | anonimo                                                                                                                                  | *     | *          |
| tema9         | anonimo                                                                                                                                  | *     | *          |
| hito-(1, 4)   | anonimo                                                                                                                                  | *     | *          |
| seq-(1, 4)    | anonimo                                                                                                                                  | *     | *          |
| c-(1, 4)      | anonimo                                                                                                                                  | *     | *          |
| e-(1, 4)      | anonimo                                                                                                                                  | *     | *          |
| clase         | anonimo                                                                                                                                  | *     | *          |
| exitosos      | anonimo                                                                                                                                  | *     | *          |
| fallidos      | anonimo                                                                                                                                  | *     | *          |
| envios        | anonimo                                                                                                                                  | *     | *          |
| videos        | anonimo                                                                                                                                  | *     | *          |
| programa      | anonimo                                                                                                                                  | *     | *          |
| sol-(1, 4)    | anonimo                                                                                                                                  | *     | *          |
| tarea1        | anonimo                                                                                                                                  | *     | *          |
| tarea2        | anonimo                                                                                                                                  | *     | *          |
| controles     | anonimo                                                                                                                                  | *     | *          |
| np            | anonimo                                                                                                                                  | *     | *          |
| examen        | anonimo                                                                                                                                  | *     | *          |
| final         | anonimo                                                                                                                                  | *     | *          |
| estado        | anonimo                                                                                                                                  | *     | *          |

### DATA_A_2021

| Campo             | Descripción                                                                                      | Tipo   | Valores       |
|-------------------|--------------------------------------------------------------------------------------------------|--------|---------------|
| hito-(1, 4)       | Son los puntajes obtenidos al realizar los ejercicios referentes de ese hito                     | int    | [1,2,3,4]     |
| seq-(1, 4)        | Secuencia de Complejidad de Ejercicios (alfabética) del Hito-n                                   | str    | *             |
| c-(1, 4)          | Complejidad Numérica Total del Hito-n                                                            | str    | *             |
| nrc               | NRC del curso                                                                                    | int    | continuos     |
| exitosos          | Número de envíos exitosos                                                                        | int    | *             |
| fallidos          | Número de envíos fallidos                                                                        | int    | *             |
| envidos           | Número total de envíos                                                                           | int    | *             |
| e-(0, 52)         | Ejercicio que realizó la persona dentro de la plataforma                                         | int    | binario [0,1] |
| programa          | ID de la carrera que está cursando el estudiante en la UNAB                                      | str    | *             |
| sol1              | Nota del estudiante al finalizar el semestre                                                     | float  | [1.0, 7.0]    |
| id                | ID del registro enviado                                                                          | int    | continuos     |

### MODULOS


In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import models.utils.visualization as utils


In [3]:
# DATA
VAR_DIR_DATA_ORIGINAL = './data/original'

# El catálogo contiene los ejercicios de programación que estaban disponibles en la página web.
df_catalogo = pd.read_csv(f"{VAR_DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")

# La variable df_data_a almacena registros generales de los estudiantes durante el semestre, es decir, datos del curso en general.
df_data_a = pd.read_csv(f"{VAR_DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")

# La variable df_data_a_2021 contiene registros globales de los estudiantes que intentaron realizar los ejercicios de la página web en 2021.
df_data_a_2021 = pd.read_csv(f"{VAR_DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")

### DIMENSIONES DE LOS DATASETS

In [ ]:
# CATALOGO
print(utils.show_dataframe_dimension(df_catalogo))
# DATA_A
print(utils.show_dataframe_dimension(df_data_a))
# DATA_A_2021
print(utils.show_dataframe_dimension(df_data_a_2021))

### INFORMACION DE LOS DATASETS

In [ ]:
# VISUALIZACION: CATALOGO
df_catalogo.head()

In [ ]:
# VISUALIZACION: DATA_A
df_data_a.head()

In [ ]:
# VISUALIZACION: DATA_A_2021
df_data_a_2021.head()

> NOTA: Se puede apreciar dos cosas:
> - En el dataset 'data_a' existen datos faltantes por proteccion a la privacidad de los alumnos y profesor
> - En el dataset 'data_a_2021' existen datos faltantes en varias filas, por lo que, se puede decir que no se pudo recopilar bien la informacion o los alumnos que interactuaron no completaron las tareas. 

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: CATALOGO
df_catalogo.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A
df_data_a.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A_2021
df_data_a_2021.info()

### DATOS FALTANTES

In [ ]:
# DATOS FALTANTES: CATALOGO
utils.show_missing_data(df_catalogo)

> NOTA: El dataset de los ejercicios (Catalogo.csv) no contempla con datos vacios.Cuenta con 53 ejercicio con sus respectivas carecteristicas.

In [ ]:
# DATOS FALTANTES: DATA_A
utils.show_missing_data(df_data_a)

> NOTA: Para el dataset 'data_a.csv' tiene multiples columnas con datos faltantes, la mayoria tienen que ver con los ejercicios que realizaron los estudiantes. Por tanto, se puede suponer que los estudiantes hacen mas ejercicios de los Hito 1 y 2, siendo que la mayoria de columnas del hito 3 y 4 tienen un 100% de datos faltantes. 

In [ ]:
# DATOS FALTANTES: DATA_A_2021
utils.show_missing_data(df_data_a_2021)

> NOTA: El dataset 'data_a_2021.csv' tienen el mismo problema que el anterior, se nota que existen semejanzas de atributos con mayores porcentajes de datos faltantes. 
> Las columnas que se repiten como Seq1, Seq2, Seq3 y Seq4, representan la secuencia de ejercicios realizados en cada hito por los estudiantes, es decir, al tener Seq3 y Seq4 todas sus datos vacios en esas columnas, se puede afirmar que los estudiantes solo realizaron ejercicos del Hito 1, en su mayoria, y algunos del Hito 2.

### METRICAS DE LOS DATASETS

In [ ]:
# DATOS ESTADISTICOS: CATALOGO
df_catalogo.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A
df_data_a.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A_2021
df_data_a_2021.describe().T

In [ ]:
# CORRELACIÓN DE VARIABLES: CATALOGO
utils.show_correlation_columns(df_catalogo, 'complexity')

> Nota: 
> Teniendo en cuenta que, si se tuviera que relacionar la complejidad del ejercicio que el estudiante realiza, su puntaje deberia estar influenciado por esta correlacion...
> Teniendo en cuenta, que las variables 'Skill' y 'Knowledge' vienen de las variables one-hot-encoded, tienen mucha correlacion, pero la complejidad esta muy influenciada por la habilidad o 'Skill' tenga el ejercicio.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A
utils.show_correlation_columns(df_data_a, 'final')

> Observacion:
> - Las variables 'final' y 'np' estan estrechamente correlacionados, esto se puede deber ya que se representan la nota final y nota de presentacion del estudiante al final del semestre.
> - Tambien se puede observar como las solemnes 4 y 3 estan igualmente correlacionadas con la nota final del estudiante. Sin embargo, esto es ilogico, ya que si los estudiantes quisieran mejorar su rendimiento academico y mejorar sus notas para eximir la asignatura, deberian realizar ejercicios de los hitos 3 y 4. Asi mismo, reducirian las probabilidades de reprobar.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A_2021
utils.show_correlation_columns(df_data_a_2021, 'sol1')

> Nota:
> - Al igual que el dataset anterior, solo se muestra ejercicios y la variable a correlacionar, en este caso, seria 'sol1' que representa el puntaje de la solemne 1 que saco el estudiante. 
> - En este dataset, solo muestra los ejercicios que mas fluyo en la obtencion del puntaje de la 'sol1'. 

### GRAFICOS

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: CATALOGO  
df_catalogo.hist(figsize=(22, 20))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A
df_data_a.hist(figsize=(30,30))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A_2021
df_data_a_2021.hist(figsize=(30, 25))
plt.show()

In [ ]:
df_catalogo = df_catalogo[['nombre', 'hito', 'skill', 'knowledge', 'complexity']].sort_values(by=["hito", "complexity"], ascending=True)
df_catalogo.head(10)

In [25]:
df_data_a = df_data_a.sort_values(by='rut', ascending=True).reset_index(drop='index')
df_data_a = df_data_a[['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1', 'sol2', 'sol3', 'sol4','estado']]

df_data_a_2021 = df_data_a_2021.sort_values(by='id', ascending=True).drop(labels=['id'], axis=1)
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0]))
df_data_a_2021 = df_data_a_2021[['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'p1', 'p2', 'p3', 'p4', 'sol1']]
df_data_a_2021.columns = ['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1']


df_data = pd.concat([df_data_a, df_data_a_2021], ignore_index=True)
df_data.columns = ['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1', 'sol2', 'sol3', 'sol4','estado']

In [ ]:
df_data.head(10)

In [ ]:
show_missing_data(df_data)

In [27]:
# FUENTE: https://programacionacademica.unab.cl/

df_data['programa'] = df_data['programa'].replace({
    'UNAB12100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22115':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22510':'INGENIERIA INDUSTRIAL',
    'UNAB12510':'INGENIERIA INDUSTRIAL',
    'UNAB22505':'INGENIERIA INDUSTRIAL',
    'UNAB21503':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB11500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB21500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB12210':'INGENIERIA CIVIL INFORMATICA',
    'UNAB18000':'BACHILLERATO EN CIENCIAS',
    'UNAB16507':'ED. CONTINUA - ECONOMIA Y NEG',
    'UNAB35636':'MAG. EN INGENIERIA INDUSTRIAL',
    'UNAB10100':'PROG. DE INTERCAMBIO',
})

In [ ]:
df_data['programa'].value_counts()

### GRAFICOS: catalogo.csv

In [ ]:
# Crear la figura y los ejes
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Ajustar la disposición de los subplots
fig.tight_layout(pad=5.0)

# Graficar el swarmplot
sns.stripplot(data=df_catalogo, x='knowledge', y='skill', hue='hito', palette="muted", dodge=True, ax=axs[0, 0])
axs[0, 0].set_title('Swarmplot: Knowledge vs Skill by Hito')

# Ocultar el cuarto subplot (axs[1, 1]) si no se usa
axs[0, 1].axis('off')

# Graficar el histplot para 'skill'
sns.histplot(data=df_catalogo, x='skill', kde=True, ax=axs[1, 0])
axs[1, 0].set_title('Histogram of Skill')

# Graficar el histplot para 'knowledge'
sns.histplot(data=df_catalogo, x='knowledge', kde=True, ax=axs[1, 1])
axs[1, 1].set_title('Histogram of Knowledge')

# Mostrar la figura
plt.show()

### GRAFICOS: df_data

In [ ]:
# Crear histogramas de todas las columnas
axes = df_data.hist(figsize=(30, 25))
# Mostrar la figura
plt.show()

In [ ]:
carreras = df_data['programa'].value_counts()
fig, ax = plt.subplots(figsize=(12, 5))  
sns.barplot(x=carreras.values, y=carreras.index, orient='h', hue=carreras.index, palette='Set2', dodge=False, ax=ax)

for container in ax.containers:
    ax.bar_label(container, fontsize=10)

plt.title('Total de estudiantes registrados por carrera', fontsize=16)
plt.xlabel('Carreras')
plt.ylabel('Número total de estudiantes participantes')
plt.show()

In [ ]:
rangos = [(1.0, 2.0), (2.0, 3.0), (3.0, 4.0), (4.0, 5.0), (5.0, 6.0), (6.0, 7.0), (7.0, 7.1)]

def contar_personas_en_rango(df, rango):
    rango_inferior, rango_superior = rango
    return df[(df['sol1'] >= rango_inferior) & (df['sol1'] < rango_superior)].shape[0]

conteo_por_rango = []
for rango in rangos:
    conteo = contar_personas_en_rango(df_data, rango)
    conteo_por_rango.append((rango, conteo))

# Preparar los datos para el gráfico
rangos_str = [f"{rango[0]:.1f}-{rango[1]:.1f}" for rango in rangos]
conteos = [conteo for rango, conteo in conteo_por_rango]

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(rangos_str, conteos, color='skyblue')

# Agregar etiquetas sobre cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom')

plt.xlabel('Rangos de sol1')
plt.ylabel('Número de personas')
plt.title('Número de personas en cada rango de sol1')
plt.xticks(rotation=45)
plt.show()

In [ ]:
programa_df = pd.DataFrame({'Aprobados': df_data[df_data['sol1'] >= 4.0].groupby('programa').size(), 'Reprobados': df_data[df_data['sol1'] < 4.0].groupby('programa').size()})
programa_df.fillna(0.0, inplace=True)
programa_df = programa_df.astype(int)   # Aseguramos que todos los valores sean enteros
y = np.arange(len(programa_df.index))   # las ubicaciones de las etiquetas
height = 0.25                           # la altura de las barras
multiplier = 0

fig, ax = plt.subplots(figsize=(14, 6))

for attribute, measurement in programa_df.to_dict().items():
    offset = height * multiplier
    rects = ax.barh(y + offset, measurement.values(), height, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Añadir algunos textos para etiquetas, título y etiquetas personalizadas del eje x, etc.
ax.set_xlabel('Cantidad')
ax.set_title('Estudiantes aprobados y reprobados por programa (Solemne 1)')
ax.set_yticks(y + height / 2)
ax.set_yticklabels(programa_df.index)
ax.legend(loc='lower right')
ax.set_xlim(0, programa_df.max().max() + 10)  # Ajuste de límites del eje x

plt.show()

## 3. PREPARACIÓN DE LOS DATOS

In [1]:
# MODULOS
import warnings
import numpy as np 
import pandas as pd
warnings.filterwarnings('ignore')

# RUTAS
DIR_DATA_ORIGINAL = './data/original'
DIR_DATA_LIMPIA = './data/cleaning'

#### EJERCICIOS (ITEMS)

In [2]:
# CARGAR DATASETS DE CATALOGO DE EJERCICIOS
# Se importa un archivo CSV que contiene información sobre un catálogo de ejercicios. 
df_catalogo = pd.read_csv(f"{DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")

# Convertir la columna 'hito' a una representación binaria de 4 bits.
# La función apply aplica la conversión solo a valores entre 1 y 4, retornando None para otros valores.
hitos = df_catalogo['hito'].apply(lambda x: format(x, '04b') if 1 <= x <= 4 else None)

# Crear columnas h1, h2, h3, h4 a partir de la representación binaria obtenida.
# Cada columna h1 a h4 representa un bit de la conversión binaria de 'hito'.
df_catalogo['h1'] = hitos.apply(lambda x: x[0] if x else None)  # Primer bit (más significativo)
df_catalogo['h2'] = hitos.apply(lambda x: x[1] if x else None)  # Segundo bit
df_catalogo['h3'] = hitos.apply(lambda x: x[2] if x else None)  # Tercer bit
df_catalogo['h4'] = hitos.apply(lambda x: x[3] if x else None)  # Cuarto bit (menos significativo)

# Crear una nueva columna 'id_ejercicio' que asigna un identificador único a cada ejercicio.
# Esto se realiza utilizando un rango basado en el número de filas en el DataFrame.
df_catalogo['id_ejercicio'] = range(len(df_catalogo))

# Reorganizar las columnas del DataFrame, manteniendo solo las columnas relevantes para el análisis.
df_catalogo = df_catalogo[['id_ejercicio', 'nombre', 'h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4', 'hito', 'skill', 'knowledge', 'complexity']]

# Crear una nueva columna 'complexity12' que representa la complejidad de cada ejercicio como un número entero.
# Se concatena la representación en string de las columnas relevantes, se rellena con ceros a la izquierda
# hasta alcanzar 12 bits, y se convierte la cadena binaria resultante a un entero.
df_catalogo['complexity12'] = (
    df_catalogo[['h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']]
    .astype(str)  # Convertir a string para concatenar
    .agg(''.join, axis=1)  # Concatenar las columnas en una sola cadena
    .apply(lambda x: int(x.zfill(12), 2))  # Convertir la cadena binaria a entero
)

# Guardar el DataFrame modificado en un nuevo archivo CSV, sin incluir el índice.
df_catalogo.to_csv(f'{DIR_DATA_LIMPIA}/catalogo.csv', index=False)

df_catalogo.head(10)

,id_ejercicio,nombre,h1,h2,h3,h4,s1,s2,s3,s4,k1,k2,k3,k4,hito,skill,knowledge,complexity,complexity12
0,0,Cálculo del dígito verificador del rut,0,0,0,1,0,0,0,1,0,1,1,1,1,1,7,23,279
1,1,Calculadora Geométrica,0,0,1,0,0,0,0,1,0,0,0,1,2,1,1,17,529
2,2,Subsecuencias de ADN,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567
3,3,Números Primos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
4,4,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257
5,5,Validador de Expresiones Matemáticas,0,1,0,0,1,0,0,0,0,1,1,1,4,8,7,135,1159
6,6,Suma de los divisores de un número,0,0,1,0,0,0,0,1,0,0,1,1,2,1,3,19,531
7,7,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257
8,8,Distancia Levenshtein,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567
9,9,Validar Secuencias de ADN,0,0,1,0,0,0,1,0,0,1,1,1,2,2,7,39,551


#### DATA_A & DATA_A_2021

In [3]:
# Se importa el archivo CSV que contiene datos de un conjunto de ejercicios (dataset_a).
df_data_a = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")

# Se importa un segundo archivo CSV que contiene datos del año 2021 (dataset_a_2021).
df_data_a_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")

# Se ordena el DataFrame df_data_a por la columna 'rut' en orden ascendente.
df_data_a = df_data_a.sort_values(by='rut', ascending=True)

# Se ordena el DataFrame df_data_a_2021 por la columna 'id' en orden ascendente.
# Luego, se elimina la columna 'id' del DataFrame.
df_data_a_2021 = df_data_a_2021.sort_values(by='id', ascending=True).drop(labels=['id'], axis=1)

# Se corrige el rango de 'rut' en df_data_a_2021 para que no se repitan.
# Los nuevos 'rut' se asignan en un rango basado en el número de filas en df_data_a.
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0]))  # Rango ajustado

# Se seleccionan columnas relevantes del DataFrame df_data_a y se renombrarán algunas de ellas para una mejor claridad.
df_data1 = df_data_a[['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1', 'sol2', 'sol3', 'sol4']]
df_data1.columns = ['id_estudiante','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1', 'sol2', 'sol3', 'sol4']

# Se seleccionan columnas relevantes del DataFrame df_data_a_2021 y se renombrarán algunas de ellas para mantener la coherencia.
df_data2 = df_data_a_2021[['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'p1', 'p2', 'p3', 'p4', 'sol1']]
df_data2.columns = ['id_estudiante', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1']

# Se reemplazan los códigos de programa por nombres descriptivos en df_data2.
df_data2['programa'] = df_data2['programa'].replace({
    'UNAB12100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22115':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22510':'INGENIERIA INDUSTRIAL',
    'UNAB12510':'INGENIERIA INDUSTRIAL',
    'UNAB22505':'INGENIERIA INDUSTRIAL',
    'UNAB21503':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB11500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB21500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB12210':'INGENIERIA CIVIL INFORMATICA',
    'UNAB18000':'BACHILLERATO EN CIENCIAS',
})

# Se eliminan estudiantes de carreras no relacionadas directamente con la asignatura 
# o que están en programas superiores al pregrado.
df_data2.drop(df_data2.query(" `programa`=='UNAB16507' or `programa`=='UNAB35636' or `programa`=='UNAB10100' ").index, inplace=True)


# Crear una copia de df_data1 y eliminar registros de estudiantes que no cumplen con los criterios de éxito.
df1 = df_data1.copy()
# Eliminar registros donde todos los hitos son menores a 1.
df1.drop(df1.query("`hito1` < 1 and `hito2` < 1 and `hito3` < 1 and `hito4` < 1").index, inplace=True)
# Eliminar registros donde los ejercicios tienen un puntaje menor a 4.0.
df1.drop(df1.query("`sol1` < 4.0 and `sol2` < 4.0 and `sol3` < 4.0 and `sol4` < 4.0").index, inplace=True)


# Procesar df_data2 de manera similar.
df2 = df_data2.copy()
# Eliminar registros donde todos los hitos son menores a 1.
df2.drop(df2.query("`hito1` < 1 and `hito2` < 1 and `hito3` < 1 and `hito4` < 1").index, inplace=True)
# Eliminar registros donde 'sol1' es menor a 4.0.
df2.drop(df2.query("`sol1` < 4.0").index, inplace=True)


# Combinar los registros rechazados de ambos DataFrames en un solo DataFrame llamado dataset_rechazadas.
dataset_rechazadas = pd.concat([df_data1.drop(df1.index), df_data2.drop(df2.index)])
# Asegurar que las columnas de hitos sean de tipo entero.
dataset_rechazadas['hito1'] = dataset_rechazadas['hito1'].astype(int)
dataset_rechazadas['hito2'] = dataset_rechazadas['hito2'].astype(int)
# Guardar los registros rechazados en un archivo CSV.
dataset_rechazadas.to_csv(f'{DIR_DATA_LIMPIA}/dataset_rechazadas.csv', index=False)


# Finalmente, todos los datos de los registros de los estudiantes se combinan en un solo DataFrame llamado dataset.
dataset = pd.concat([df1, df2])
# Asegurar que las columnas de hitos sean de tipo entero.
dataset['hito1'] = dataset['hito1'].astype(int)
dataset['hito2'] = dataset['hito2'].astype(int)
# Guardar el DataFrame combinado en un archivo CSV.
dataset.to_csv(f'{DIR_DATA_LIMPIA}/dataset.csv', index=False)

In [4]:
# Mostrar las primeras filas del DataFrame combinado para verificar el contenido.
dataset.head()

,id_estudiante,programa,hito1,hito2,hito3,hito4,exitosos,fallidos,ejercicios_hito1,ejercicios_hito2,ejercicios_hito3,ejercicios_hito4,sol1,sol2,sol3,sol4
283,0,INGENIERIA INDUSTRIAL,18,3,0,0,15,24,:25:17:4:26:29:23:0:22:10:3:42:18,:6:1:44,NaN,NaN,6.2,6.8,5.1,6.0
393,1,INGENIERIA CIVIL INDUSTRIAL,14,0,0,0,10,6,:25:17:4:26:29:23:0:22:10:3,NaN,NaN,NaN,7.0,6.9,7.0,7.0
99,2,INGENIERIA CIVIL INDUSTRIAL,17,0,0,0,11,20,:25:17:4:26:29:23:3:42:32:18:35,NaN,NaN,NaN,5.6,6.0,4.5,5.4
59,3,INGENIERIA CIVIL INDUSTRIAL,1,0,0,0,7,17,:25,NaN,NaN,NaN,3.9,4.7,3.7,4.1
284,4,INGENIERIA EN COMPUTACION E INFORMATICA,19,2,0,0,13,18,:25:17:4:29:23:0:22:3:42:32:13,:1:44,NaN,NaN,4.6,6.6,4.7,5.3


In [5]:
# DATASET RECHAZADA
dataset_rechazadas.head()

,id_estudiante,programa,hito1,hito2,hito3,hito4,exitosos,fallidos,ejercicios_hito1,ejercicios_hito2,ejercicios_hito3,ejercicios_hito4,sol1,sol2,sol3,sol4
0,5,INGENIERIA EN COMPUTACION E INFORMATICA,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,7,INGENIERIA CIVIL INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2,16,INGENIERIA INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
109,34,INGENIERIA CIVIL INFORMATICA,0,0,0,0,0,31,NaN,NaN,NaN,NaN,6.2,5.5,5.6,5.8
3,44,INGENIERIA CIVIL INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0


#### PRUEBA DIAGNOSTICO (DATA_A)

In [6]:
# Filtrar y crear un nuevo DataFrame con columnas específicas del DataFrame original 'df_data_a'
df_prueba_diagnostico = df_data_a[[
    't1', 'r1', 'p1',
    't2', 'r2', 'p2',
    't3', 'r3', 'p3',
    't4', 'r4', 'p4',
    't5', 'r5', 'p5',
    't6', 'r6', 'p6',
    'op1', 'op2', 'op3', 'op4', 'op5', 'op6',
    'score', 'score_a', 'score_p', 'score_d', 'score_s'
]]

# Agregar una nueva columna 'id_estudiante' al DataFrame 'df_prueba_diagnostico' utilizando el identificador 'rut' del DataFrame original
df_prueba_diagnostico['id_estudiante'] = df_data_a['rut']

df_prueba_diagnostico = df_prueba_diagnostico[[
    'id_estudiante',
    't1', 'r1', 'p1',
    't2', 'r2', 'p2',
    't3', 'r3', 'p3',
    't4', 'r4', 'p4',
    't5', 'r5', 'p5',
    't6', 'r6', 'p6',
    'op1', 'op2', 'op3', 'op4', 'op5', 'op6',
    'score', 'score_a', 'score_p', 'score_d', 'score_s'
]]

# Guardar el nuevo DataFrame 'df_prueba_diagnostico' en un archivo CSV
df_prueba_diagnostico.to_csv(f'{DIR_DATA_LIMPIA}/prueba_diagnostico.csv', index=False)  # Sin incluir índices

# Mostrar las primeras cinco filas del DataFrame 'df_prueba_diagnostico' para verificar su contenido
df_prueba_diagnostico.head()

,id_estudiante,t1,r1,p1,t2,r2,p2,t3,r3,p3,...,op2,op3,op4,op5,op6,score,score_a,score_p,score_d,score_s
283,0,64.301310,1,4,251.056409,3,10,224.745993,1,19,...,1,0,1,1,0,8,5,4,2,3
393,1,97.170900,2,5,93.273640,2,9,108.991540,1,18,...,0,0,1,0,1,8,4,4,2,3
99,2,85.596734,1,8,217.071180,7,16,150.406520,1,11,...,1,0,0,0,1,8,5,4,3,2
59,3,57.776163,1,2,97.665937,1,4,187.063634,2,11,...,0,0,0,1,0,6,3,3,1,2
284,4,32.519816,1,3,57.514397,2,5,75.568439,1,8,...,1,1,0,1,1,10,6,4,3,3


#### DATASET

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766 entries, 283 to 838
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_estudiante     766 non-null    int64  
 1   programa          766 non-null    object 
 2   hito1             766 non-null    int64  
 3   hito2             766 non-null    int64  
 4   hito3             766 non-null    int64  
 5   hito4             766 non-null    int64  
 6   exitosos          766 non-null    int64  
 7   fallidos          766 non-null    int64  
 8   ejercicios_hito1  763 non-null    object 
 9   ejercicios_hito2  359 non-null    object 
 10  ejercicios_hito3  0 non-null      float64
 11  ejercicios_hito4  0 non-null      float64
 12  sol1              766 non-null    float64
 13  sol2              424 non-null    float64
 14  sol3              424 non-null    float64
 15  sol4              424 non-null    float64
dtypes: float64(6), int64(7), object(3)
memory usage

> NOTA:

> Los datos que se utlizaran para los experimentos y creacion del sistema de recomendacion seran los mostrados en el 'dataset'. Sin embargo, no todos los datos seran utiles para entrenar un recomendador de ejecicios, ya que existen algunas anomalias en los registros de los estudiantes que puede que afecte a las recomendaciones del modelo. Por ejemplo: estudiantes que no realizaron nada durante la asignatura y reprobaron; estudiantes que no realizaron nada durante la asignatura y aprobaron; estudiantes que hicieron pocos o varios ejercicios y reprobaron la asignatura. 

> Por tanto, se agrupara los registros de estudiantes con un minimo de exito para usarlo y medir como reacciona con aquellos que tuvieron poco exito durante la asignatura. 

#### MATRICES DE FACTORIZACION

In [8]:
from models.utils.transformation import *

In [9]:
# Llamada a la función
matrix_factorization = create_factorization_matrix(dataset, df_catalogo, user_col='id_estudiante', items_col=['ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4'])
# Llamada a la función
matrix_factorization_rechadas = create_factorization_matrix(dataset_rechazadas, df_catalogo, user_col='id_estudiante', items_col=['ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4'])


matrix_factorization.to_csv(f'{DIR_DATA_LIMPIA}/mf_dataset.csv', index=False)
matrix_factorization_rechadas.to_csv(f'{DIR_DATA_LIMPIA}/mf_rechazadas.csv', index=False)


In [10]:
# Mostrar la matriz de factorización resultante
matrix_factorization

,id_estudiante,e0,e1,e2,e3,e4,e5,e6,e7,e8,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,0,1,1,0,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,1301,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
762,1302,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
763,1303,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
764,1304,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Mostrar la matriz de factorización resultante
matrix_factorization_rechadas

,id_estudiante,e0,e1,e2,e3,e4,e5,e6,e7,e8,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,1272,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
533,1283,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
534,1287,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
535,1288,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 4. MODELAMIENTO

## 5. EVALUCIÓN

## 6. IMPLEMENTACIÓN